In [2]:
import datetime
import pandas as pd
import numpy as np
import gdn_db
import gdn_datacollector
import gdn_controller as controller
import gdn_gsn_ai_behavior_log as logger
from gdn_gsn_ai_behavior_log import BehaviorType
import bid_operator
import json
import math

DATADASE = "dev_gdn"
START_TIME = 'start_time'
STOP_TIME = 'stop_time'
AI_START_DATE = 'ai_start_date'
AI_STOP_DATE = 'ai_stop_date'
AD_ID = 'ad_id'
ADGROUP_ID = 'adgroup_id'
CAMPAIGN_ID = 'campaign_id'
CHARGE = 'charge'
TARGET = 'target'
BID_AMOUNT = 'bid_amount'
REQUEST_TIME = 'request_time'
TARGET_LEFT = 'target_left'

INIT_BID = 'init_bid'
LAST_BID = 'last_bid'
ADGROUP_PROGRESS = 'adgroup_progress'
CAMPAIGN_PROGRESS = 'campaign_progress'
BIDDING_INDEX = {
    'cpc': 'cpc_bid',
    'Target CPA': 'cpa_bid',
}
DESTINATION_INDEX = {
    'cpc': 'clicks',
    'cpa': 'conversions',
    'LINK_CLICKS': 'clicks',
    'CONVERSIONS':'conversions',
}
class CampaignAdapter(object):
    def __init__(self, campaign_id):
        self.mydb = gdn_db.connectDB( DATADASE )
        self.limit = 9000
        self.hour_per_day = 24
        self.campaign_id = campaign_id
        self.request_time = datetime.datetime.now()
        self.time_progress = ( self.request_time.hour + 1 ) / self.hour_per_day
        self.init_bid_dict = dict()
        self.last_bid_dict = dict()
        
    def _get_df(self):
        campaign_sql = "SELECT * FROM campaign_target WHERE campaign_id={}".format( self.campaign_id )
        adgroup_sql = "select * from adgroup_insights WHERE campaign_id = {} AND DATE(request_time) = '{}'".format( self.campaign_id, self.request_time.date() )
        self.df_camp = pd.read_sql( campaign_sql, con=self.mydb )
        self.df_adgroup = pd.read_sql( adgroup_sql, con=self.mydb )
        return
    
    def _get_bid(self):
        df_init_bid = pd.read_sql( "SELECT * FROM adgroup_initial_bid WHERE campaign_id={} ;".format( self.campaign_id ), con=self.mydb )
        self.get_adgroup_list()
        bid_amount_type = BIDDING_INDEX[ self.df_adgroup['bidding_type'].iloc[0] ]
        for adgroup in self.adgroup_list:
            if len(self.df_adgroup[self.df_adgroup.adgroup_id==adgroup]) != 0:
                init_bid = df_init_bid[BID_AMOUNT][df_init_bid.adgroup_id==adgroup].head(1).iloc[0].astype(dtype=object)
                last_bid = self.df_adgroup[ bid_amount_type ][self.df_adgroup.adgroup_id==adgroup].tail(1).iloc[0].astype(dtype=object)
                self.init_bid_dict.update({ adgroup: init_bid })
                self.last_bid_dict.update({ adgroup: last_bid })
        return
    
    def get_periods_left(self):
        self.periods_left = 0
        try:
            self.periods_left = ( self.df_camp[ AI_STOP_DATE ].iloc[0] - self.request_time.date() ).days + 1
        except:
            self.periods_left = ( datetime.datetime.now().date() - self.request_time.date() ).days + 1
        finally:
            return self.periods_left
    
    def get_periods(self):
        try:
            self.periods = ( self.df_camp[ AI_STOP_DATE ].iloc[0] - self.df_camp[ AI_START_DATE ].iloc[0] ).days
        except:
            self.periods = ( datetime.datetime.now() - self.df_camp[ AI_START_DATE ].iloc[0] ).days
        return self.periods
    
    def get_campaign_performance(self):
        self.campaign_performance = self.df_camp[ TARGET ].div(self.df_camp[ 'period' ] ).sum()
        return self.campaign_performance
    
    def get_campaign_target(self):
        self.campaign_target = self.df_camp[ TARGET_LEFT ].iloc[0].astype(dtype=object)
        return self.campaign_target
    
    def get_campaign_day_target(self):
        self.campaign_day_target = self.campaign_target / self.periods_left
        return self.campaign_day_target

    def get_campaign_progress(self):
        self.campaign_progress = self.campaign_performance / self.campaign_day_target
        self.campaign_progress = 1 if self.campaign_day_target <= 0 else self.campaign_progress
        return self.campaign_progress
    
    def get_adgroup_list(self):
        adgroup_list_sql = "select DISTINCT adgroup_id from (select * from adgroup_insights WHERE campaign_id = {} and status='enabled' order by request_time) as a group by adgroup_id".format( self.campaign_id )
        self.mycursor = self.mydb.cursor()
        self.mycursor.execute( adgroup_list_sql )
        default = self.mycursor.fetchall()
        self.adgroup_list = [ i[0] for i in default ]
        return self.adgroup_list
    
    def retrieve_campaign_attribute(self):
        self._get_df()
        self.get_adgroup_list()
        self._get_bid()
        self.get_periods_left()
        self.get_periods()
        self.get_campaign_performance()
        self.get_campaign_target()
        self.get_campaign_day_target()
        self.get_campaign_progress()
        self.mydb.close()
        return

class AdGroupAdapter(CampaignAdapter):
    def __init__(self, adgroup_id, camp):
        self.mydb = gdn_db.connectDB( DATADASE )
        self.adgroup_id = adgroup_id
        self.camp = camp

    def init_campaign(self, camp):
        self.time_progress = camp.time_progress
        self.limit = camp.limit
        self.hour_per_day = camp.hour_per_day
        self.request_time = camp.request_time
        self.df_adgroup = camp.df_adgroup
        self.df_camp = camp.df_camp
        self.init_bid_dict = camp.init_bid_dict
        self.last_bid_dict = camp.last_bid_dict
        self.periods_left = camp.periods_left
        self.periods = camp.periods
        self.campaign_performance = camp.campaign_performance
        self.campaign_target = camp.campaign_target
        self.campaign_day_target = camp.campaign_day_target
        self.campaign_progress = camp.campaign_progress
        return
    
    def get_campaign_id(self):
        print(self.df_adgroup[ CAMPAIGN_ID ].iloc[0].astype(dtype=object))
        self.campaign_id = self.df_adgroup[ CAMPAIGN_ID ].iloc[0].astype(dtype=object)
        return self.campaign_id
    
    def get_adgroup_day_target(self):
        adgroup_num = len( self.camp.adgroup_list )
        self.adgroup_day_target = self.camp.campaign_day_target / adgroup_num
        return self.adgroup_day_target
    
    def get_adgroup_performance(self):
        try:
            target_performance_index = DESTINATION_INDEX[self.df_camp['destination_type'].iloc[0]]
            self.adgroup_performance = self.df_adgroup[self.df_adgroup.adgroup_id==self.adgroup_id][[ target_performance_index ]].tail(1).iloc[0,0]
        except Exception as e:
            print('[facebook_adapter.AdGroupAdapter.get_adgroup_performance()]', e)
            self.adgroup_performance = 0
        if math.isnan(self.adgroup_performance):
            self.adgroup_performance = 0
        return self.adgroup_performance
    
    def get_bid(self):
        self.init_bid = self.init_bid_dict[self.adgroup_id]
        self.last_bid = self.last_bid_dict[self.adgroup_id]
        return
    
    def get_adgroup_time_target(self):
        self.adgroup_time_target = self.adgroup_day_target * self.time_progress
        return self.adgroup_time_target
    
    def get_adgroup_progress(self):
#         print(self.adgroup_performance, self.adgroup_time_target)
        self.adgroup_progress = self.adgroup_performance / self.adgroup_time_target
        self.adgroup_progress = 1 if self.adgroup_time_target <= 0 else self.adgroup_progress
        return self.adgroup_progress
    
    def retrieve_adgroup_attribute(self):
        self.init_campaign(self.camp)
        self.get_campaign_id()
        self.get_adgroup_day_target()
        self.get_adgroup_performance()
        self.get_bid()
        self.get_adgroup_time_target()
        self.get_adgroup_progress()
        self.mydb.close()
        return {
            ADGROUP_ID:self.adgroup_id,
            INIT_BID:self.init_bid,
            LAST_BID:self.last_bid,
            ADGROUP_PROGRESS:self.adgroup_progress,
            CAMPAIGN_PROGRESS:self.campaign_progress
        }

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
def main():
    start_time = datetime.datetime.now()
    campaign_dict_list = gdn_db.get_campaign().to_dict('records')
    for campaign_dict in campaign_dict_list:
        campaign_id = campaign_dict['campaign_id']
        destination_type = campaign_dict['destination_type']
        account_id = campaign_dict['customer_id']
        
        result={ 'media': 'GDN', 'campaign_id': campaign_id, 'contents':[] }
        print('[campaign_id]: ', campaign_id)
        
        camp = CampaignAdapter( campaign_id )
        camp.retrieve_campaign_attribute()
        adgroup_list = camp.adgroup_list

        for adgroup in adgroup_list:
            try:
                ad_group_adapter = AdGroupAdapter( adgroup, camp )
                
                status_dict = ad_group_adapter.retrieve_adgroup_attribute()
                print(status_dict)
                media = result['media']
                bid_dict = bid_operator.adjust(media, **status_dict)
                
                ad_group_pair = {
                    'db_type': 'dev_gdn', 'campaign_id': campaign_id, 'adgroup_id': adgroup,
                    'criterion_id': None, 'criterion_type': 'adgroup'
                }
                logger.save_adgroup_behavior(
                    behavior_type=BehaviorType.ADJUST, behavior_misc=bid_dict['bid'], **ad_group_pair)
                result['contents'].append(bid_dict)
                
                gdn_datacollector.update_adgroup_bid(account_id, adgroup, bid_dict['bid'])
                del s
            except Exception as e:
                print('[facebook_adapter.AdGroupAdapter] update unavailable: ', e)
        
        mydict_json = json.dumps(result, cls=MyEncoder)
        release_json = json.dumps(release_version_result)
        gdn_db.insert_result( campaign_id, mydict_json )
        del camp
#         except:
#             print('pass')
#             pass
        
#     campaign_id = 1747836664
#     result={ 'media': 'GDN', 'campaign_id': campaign_id, 'contents':[] }
#     camp = CampaignAdapter( campaign_id )
#     camp.retrieve_campaign_attribute()
#     adgroup_list = camp.get_adgroup_list()
#     for adgroup in adgroup_list:
#         ad_group_adapter = AdGroupAdapter( adgroup, camp )
#         status = ad_group_adapter.retrieve_adgroup_attribute()
#         media = result['media']
#         bid = bid_operator.adjust(media, **status)
#         result['contents'].append(bid)
#         print(result)
#         del s
#     del camp
    
    print(datetime.datetime.now()-start_time)
    return
    


In [2]:
#adapter = CampaignAdapter(1985403837)

In [3]:
if __name__=='__main__':
    main()
    import gc
    gc.collect()


1837064050
1837064050
{'adgroup_id': 70749935438, 'init_bid': 3.3, 'last_bid': 3.3, 'adgroup_progress': 1, 'campaign_progress': 1}
====Test====
3.3 3.3
====Behavior Log====
70749935438 None 3.3 <class 'float'> 3.3 <class 'numpy.float64'>
1837064050
{'adgroup_id': 70749935478, 'init_bid': 3.3, 'last_bid': 3.3, 'adgroup_progress': 1, 'campaign_progress': 1}
====Test====
3.3 3.3
====Behavior Log====
70749935478 None 3.3 <class 'float'> 3.3 <class 'numpy.float64'>
1837064050
{'adgroup_id': 70749935518, 'init_bid': 3.3, 'last_bid': 3.3, 'adgroup_progress': 1, 'campaign_progress': 1}
====Test====
3.3 3.3
====Behavior Log====
70749935518 None 3.3 <class 'float'> 3.3 <class 'numpy.float64'>
1837064050
{'adgroup_id': 70749935678, 'init_bid': 3.3, 'last_bid': 3.3, 'adgroup_progress': 1, 'campaign_progress': 1}
====Test====
3.3 3.3
====Behavior Log====
70749935678 None 3.3 <class 'float'> 3.3 <class 'numpy.float64'>
1837064050
{'adgroup_id': 75201413243, 'init_bid': 4.0, 'last_bid': 4.0, 'adgroup

====Behavior Log====
75487476135 None 4.2 <class 'float'> 4.2 <class 'numpy.float64'>
2053536668
{'adgroup_id': 75487476175, 'init_bid': 3.0, 'last_bid': 3.41, 'adgroup_progress': 0.9285714285714286, 'campaign_progress': 1.176681783824641}
====Test====
3.41 3.41
====Behavior Log====
75487476175 None 3.41 <class 'float'> 3.41 <class 'numpy.float64'>
2053536668
{'adgroup_id': 75487476215, 'init_bid': 3.0, 'last_bid': 3.0, 'adgroup_progress': 2.785714285714286, 'campaign_progress': 1.176681783824641}
====Test====
3.0 3
====Behavior Log====
75487476215 None 3.0 <class 'float'> 3 <class 'numpy.int64'>
2053536668
{'adgroup_id': 75487476375, 'init_bid': 3.0, 'last_bid': 3.41, 'adgroup_progress': 0.9285714285714286, 'campaign_progress': 1.176681783824641}
====Test====
3.41 3.41
====Behavior Log====
75487476375 None 3.41 <class 'float'> 3.41 <class 'numpy.float64'>
2053536668
{'adgroup_id': 75487476415, 'init_bid': 3.0, 'last_bid': 4.2, 'adgroup_progress': 0.0, 'campaign_progress': 1.1766817838

In [4]:
#!jupyter nbconvert --to script gdn_adapter.ipynb

In [11]:
campaign_list = gdn_db.get_campaign().to_dict('records')

In [12]:
[campaign for campaign in campaign_list]

[{'customer_id': 6714857152,
  'campaign_id': 2053556135,
  'channel_type': 'Display',
  'status': 'enabled',
  'ai_status': None,
  'destination_type': 'LINK_CLICKS',
  'is_optimized': 'False',
  'optimized_date': datetime.date(2019, 8, 2),
  'cost_per_target': 3.94138,
  'daily_budget': 100.0,
  'daily_target': 22.5556,
  'destination': 1150.0,
  'period': 46,
  'period_left': 18,
  'spend': 2932.38,
  'ai_spend_cap': 4600.0,
  'ai_start_date': datetime.date(2019, 7, 5),
  'ai_stop_date': datetime.date(2019, 8, 19),
  'spend_cap': None,
  'start_time': Timestamp('2019-07-05 00:00:00'),
  'stop_time': Timestamp('2019-08-19 00:00:00'),
  'target': 744,
  'target_left': 406,
  'bidding_type': 'cpc',
  'impressions': 248119,
  'ctr': 0.3,
  'clicks': 744,
  'conversions': 2,
  'cost_per_click': 3.94138,
  'cost_per_conversion': 1466.19,
  'all_conversions': 2.0,
  'cost_per_all_conversion': 1466.19},
 {'customer_id': 2132755743,
  'campaign_id': 1755842283,
  'channel_type': 'Display',
 